In [ ]:
PROJECT_DIR = '/workspace/current/loce'

import sys
sys.path.append(PROJECT_DIR)

from bg_randomized_loce.utils.loce_storage_helpers import all_pkls_to_npz
from bg_randomized_loce.utils.eval_util import with_globalized_ces
import pandas as pd
pd.set_option('display.max_rows', 1000)
import numpy as np

bg_randomized_loce.utils.consts.PROJECT_DIR = PROJECT_DIR


## Load the concept vectors with meta info

In [ ]:
results_dir = os.path.join(PROJECT_DIR, 'results')
npz_dir = os.path.join(PROJECT_DIR, 'data/results/npz')

ce_infos = await all_pkls_to_npz(results_dir=results_dir, npz_dir=npz_dir)#, force_update=True)
assert len(ce_infos) > 0
# to dataframe
orig_df = pd.DataFrame(ce_infos)
df: pd.DataFrame = with_globalized_ces(orig_df.copy(), experiment_setting_cols=EXPERIMENT_SETTING_COLS, add_depth=True)

## Global CEs
df_glo = df[df[GLO_OR_LOC] == GLOBAL]
## Local CEs
df_loc = df[df[GLO_OR_LOC] == LOCAL]
## Local-to-global CEs via averaging
df_loglo = df[df[GLO_OR_LOC] == LOC_TO_GLOB]

# display some simple stats
display(df.columns)  # what columns exist?
for col in [*EXPERIMENT_SETTING_COLS, *DERIVED_SETTINGS]: print(col, df[col].unique())

nans = df[df[CE].apply(lambda x: np.any(np.isnan(x)))]
print('Total entries:', len(df.index))  # how many entries in total?
print('NaN values:', len(nans.index))  # how may of these contain vectors with NaN entries?

## Move all npz files into a faster filesystem
```python
import shutil
npz_paths = tqdm(list(Path(results_dir).glob('**/*.npz')))
for npz_path in npz_paths:
    npz_newpath = Path(PROJECT_DIR)/'data/results/npz'/npz_path.relative_to(results_dir)
    os.makedirs(os.path.dirname(npz_newpath), exist_ok=True)
    shutil.copy2(npz_path, npz_newpath)
```

# Experiment plan

1. Similarity:
   - [ ] Global, local-to-global: Cosine Similarity (TO RUN)
   - ~~Local: Wasserstein distance (TO IMPLEMENT)~~
2. Generalization (only global, local-to-global): IoU drop from vanilla to bg randomized test data
   - [X] mean IoU with/without training-time randomization
3. Background bias: Mean IoU drop for concept-background pairs
   - [X] mean IoU for concept-background pairs; relative drop
